# Real-Time Stop Sign Identification using Contour Detection
### Olivia Roberts, Anne Konicki, William Grieder

**Image Preprocessing**

In [27]:
#imports

import cv2

In [28]:
# Convert image to HSV
bgr_img = cv2.imread('./assets/StopSign02.jpg')
hsv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)

cv2.imshow('Original', bgr_img)
cv2.imshow('HSV', hsv_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
# Binarize Image

# Best threshold appears to be 192. More testing needed

thresholds = [64, 128, 192]
for thresh in thresholds:
    _, bin_img = cv2.threshold(hsv_img, thresh, 255, cv2.THRESH_BINARY)

    cv2.imshow(f'Binarized: {thresh}', bin_img)

_, bin_img = cv2.threshold(hsv_img, 192, 255, cv2.THRESH_BINARY) # Store the 192 threshold version of the image for later use
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
# Apply Gaussian Blur

### TODO: Apply a Gaussian Blur to the image after binarization.
# Analyze the images to see if the stop sign's become more visually distinct on the image.
# Use different threshold values as well. StopSign2 will be a useful image for this as part of the sign is obscured.
# See if applying the blur can potentially remove the graffiti affect. it may also be useful to apply the blur to detect ellipses rather than octagons.
# Blurring the image could make contour detection easier



## Contour Detection

In [ ]:
# Ellipse/Octagonal contour detection

# TODO: Discover an octagonal/elliptical contour in the image

In [ ]:
# Bounding Box

# TODO: Draw a bounding box around the specific region with the discovered shape should it be found